In [1]:
pass

In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor

# Load the crime dataset
crime_df = pd.read_csv('../Dataset/Chicago_Crimes_2012_to_2017.csv')
crime_df.head()


,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,3,10508693,HZ250496,05/03/2016 11:40:00 PM,013XX S SAWYER AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,...,24.0,29.0,08B,1154907.0,1893681.0,2016,05/10/2016 03:56:50 PM,41.864073,-87.706819,"(41.864073157, -87.706818608)"
1,89,10508695,HZ250409,05/03/2016 09:40:00 PM,061XX S DREXEL AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,...,20.0,42.0,08B,1183066.0,1864330.0,2016,05/10/2016 03:56:50 PM,41.782922,-87.604363,"(41.782921527, -87.60436317)"
2,197,10508697,HZ250503,05/03/2016 11:31:00 PM,053XX W CHICAGO AVE,0470,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,STREET,False,...,37.0,25.0,24,1140789.0,1904819.0,2016,05/10/2016 03:56:50 PM,41.894908,-87.758372,"(41.894908283, -87.758371958)"
3,673,10508698,HZ250424,05/03/2016 10:10:00 PM,049XX W FULTON ST,0460,BATTERY,SIMPLE,SIDEWALK,False,...,28.0,25.0,08B,1143223.0,1901475.0,2016,05/10/2016 03:56:50 PM,41.885687,-87.749516,"(41.885686845, -87.749515983)"
4,911,10508699,HZ250455,05/03/2016 10:00:00 PM,003XX N LOTUS AVE,0820,THEFT,$500 AND UNDER,RESIDENCE,False,...,28.0,25.0,06,1139890.0,1901675.0,2016,05/10/2016 03:56:50 PM,41.886297,-87.761751,"(41.886297242, -87.761750709)"


In [3]:
column_values = crime_df['Primary Type'].unique()
print(column_values)

['BATTERY' 'PUBLIC PEACE VIOLATION' 'THEFT' 'WEAPONS VIOLATION' 'ROBBERY'
 'MOTOR VEHICLE THEFT' 'ASSAULT' 'OTHER OFFENSE' 'DECEPTIVE PRACTICE'
 'CRIMINAL DAMAGE' 'CRIMINAL TRESPASS' 'BURGLARY' 'STALKING'
 'CRIM SEXUAL ASSAULT' 'NARCOTICS' 'SEX OFFENSE' 'OBSCENITY'
 'OFFENSE INVOLVING CHILDREN' 'KIDNAPPING' 'HOMICIDE'
 'INTERFERENCE WITH PUBLIC OFFICER' 'PROSTITUTION' 'GAMBLING'
 'INTIMIDATION' 'ARSON' 'LIQUOR LAW VIOLATION' 'NON-CRIMINAL'
 'PUBLIC INDECENCY' 'HUMAN TRAFFICKING'
 'CONCEALED CARRY LICENSE VIOLATION' 'NON - CRIMINAL'
 'OTHER NARCOTIC VIOLATION' 'NON-CRIMINAL (SUBJECT SPECIFIED)']


In [4]:
crime_weightage = {
    'BATTERY': 7, 
    'PUBLIC PEACE VIOLATION': 2, 
    'THEFT': 2, 
    'WEAPONS VIOLATION': 6,
    'ROBBERY': 5,
    'MOTOR VEHICLE THEFT': 4,
    'ASSAULT': 6,
    'OTHER OFFENSE': 3,
    'DECEPTIVE PRACTICE': 3,
    'CRIMINAL DAMAGE': 4, 
    'CRIMINAL TRESPASS': 2,
    'BURGLARY': 4, 
    'STALKING': 5,
    'CRIM SEXUAL ASSAULT': 9, 
    'NARCOTICS': 3,
    'SEX OFFENSE': 8, 
    'OBSCENITY': 5,
    'OFFENSE INVOLVING CHILDREN': 8, 
    'KIDNAPPING': 8, 
    'HOMICIDE': 10,
    'INTERFERENCE WITH PUBLIC OFFICER': 4,
    'PROSTITUTION': 2, 
    'GAMBLING': 2,
    'INTIMIDATION': 4,
    'ARSON': 6, 
    'LIQUOR LAW VIOLATION': 2, 
    'NON-CRIMINAL': 1,
    'PUBLIC INDECENCY': 4, 
    'HUMAN TRAFFICKING': 10,
    'CONCEALED CARRY LICENSE VIOLATION': 4,
    'NON - CRIMINAL': 1,
    'OTHER NARCOTIC VIOLATION': 3, 
    'NON-CRIMINAL (SUBJECT SPECIFIED)': 1
}

In [5]:
frequency_df = crime_df[['Location','Primary Type']].copy()
frequency_df.head()

,Location,Primary Type
0,"(41.864073157, -87.706818608)",BATTERY
1,"(41.782921527, -87.60436317)",BATTERY
2,"(41.894908283, -87.758371958)",PUBLIC PEACE VIOLATION
3,"(41.885686845, -87.749515983)",BATTERY
4,"(41.886297242, -87.761750709)",THEFT


In [6]:
frequency_df['crime_freq'] = frequency_df['Primary Type'].apply(lambda y: crime_weightage[y])

In [7]:
frequency_df.head()

,Location,Primary Type,crime_freq
0,"(41.864073157, -87.706818608)",BATTERY,7
1,"(41.782921527, -87.60436317)",BATTERY,7
2,"(41.894908283, -87.758371958)",PUBLIC PEACE VIOLATION,2
3,"(41.885686845, -87.749515983)",BATTERY,7
4,"(41.886297242, -87.761750709)",THEFT,2


In [8]:
frequency_df = frequency_df.groupby(['Location', 'Primary Type'])['crime_freq'].sum().reset_index()

In [9]:
frequency_df.head()

,Location,Primary Type,crime_freq
0,"(36.619446395, -91.686565684)",ASSAULT,30
1,"(36.619446395, -91.686565684)",BATTERY,112
2,"(36.619446395, -91.686565684)",BURGLARY,8
3,"(36.619446395, -91.686565684)",CRIMINAL DAMAGE,24
4,"(36.619446395, -91.686565684)",CRIMINAL TRESPASS,8


In [10]:
output_df = frequency_df.pivot(index='Location', columns='Primary Type', values='crime_freq').fillna(0).reset_index()

In [11]:
output_df.head(20)

Primary Type,Location,ARSON,ASSAULT,BATTERY,BURGLARY,CONCEALED CARRY LICENSE VIOLATION,CRIM SEXUAL ASSAULT,CRIMINAL DAMAGE,CRIMINAL TRESPASS,DECEPTIVE PRACTICE,...,OTHER NARCOTIC VIOLATION,OTHER OFFENSE,PROSTITUTION,PUBLIC INDECENCY,PUBLIC PEACE VIOLATION,ROBBERY,SEX OFFENSE,STALKING,THEFT,WEAPONS VIOLATION
0,"(36.619446395, -91.686565684)",0.0,30.0,112.0,8.0,0.0,0.0,24.0,8.0,12.0,...,0.0,18.0,0.0,0.0,4.0,15.0,0.0,0.0,20.0,6.0
1,"(41.644585429, -87.616512829)",0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"(41.644589713, -87.61587983)",0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
3,"(41.644604096, -87.610728247)",0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
4,"(41.644606368, -87.606301217)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,"(41.644606925, -87.608997659)",0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,"(41.644607723, -87.613055128)",0.0,6.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0
7,"(41.644609763, -87.613380728)",0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,"(41.644612024, -87.601570479)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
9,"(41.644612187, -87.601596088)",0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
output_df['Frequency'] = output_df.iloc[:, 1:].sum(axis=1)
output_df.head(20)

Primary Type,Location,ARSON,ASSAULT,BATTERY,BURGLARY,CONCEALED CARRY LICENSE VIOLATION,CRIM SEXUAL ASSAULT,CRIMINAL DAMAGE,CRIMINAL TRESPASS,DECEPTIVE PRACTICE,...,OTHER OFFENSE,PROSTITUTION,PUBLIC INDECENCY,PUBLIC PEACE VIOLATION,ROBBERY,SEX OFFENSE,STALKING,THEFT,WEAPONS VIOLATION,Frequency
0,"(36.619446395, -91.686565684)",0.0,30.0,112.0,8.0,0.0,0.0,24.0,8.0,12.0,...,18.0,0.0,0.0,4.0,15.0,0.0,0.0,20.0,6.0,322.0
1,"(41.644585429, -87.616512829)",0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
2,"(41.644589713, -87.61587983)",0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,9.0
3,"(41.644604096, -87.610728247)",0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,8.0
4,"(41.644606368, -87.606301217)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
5,"(41.644606925, -87.608997659)",0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
6,"(41.644607723, -87.613055128)",0.0,6.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,50.0
7,"(41.644609763, -87.613380728)",0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
8,"(41.644612024, -87.601570479)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0
9,"(41.644612187, -87.601596088)",0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0


In [13]:
output_df = output_df[['Location','Frequency']].copy()

In [14]:
output_df.head(20)

Primary Type,Location,Frequency
0,"(36.619446395, -91.686565684)",322.0
1,"(41.644585429, -87.616512829)",6.0
2,"(41.644589713, -87.61587983)",9.0
3,"(41.644604096, -87.610728247)",8.0
4,"(41.644606368, -87.606301217)",3.0
5,"(41.644606925, -87.608997659)",4.0
6,"(41.644607723, -87.613055128)",50.0
7,"(41.644609763, -87.613380728)",4.0
8,"(41.644612024, -87.601570479)",2.0
9,"(41.644612187, -87.601596088)",7.0


In [15]:
crime_df = output_df.copy()
crime_df.head()

Primary Type,Location,Frequency
0,"(36.619446395, -91.686565684)",322.0
1,"(41.644585429, -87.616512829)",6.0
2,"(41.644589713, -87.61587983)",9.0
3,"(41.644604096, -87.610728247)",8.0
4,"(41.644606368, -87.606301217)",3.0


In [16]:
crime_df['Location'].dtype

dtype('O')

In [17]:
crime_df['Location'] = crime_df['Location'].apply(lambda x: tuple(map(float, x.strip('()').split(', '))))


In [18]:
crime_df['Location'].head()

0    (36.619446395, -91.686565684)
1    (41.644585429, -87.616512829)
2     (41.644589713, -87.61587983)
3    (41.644604096, -87.610728247)
4    (41.644606368, -87.606301217)
Name: Location, dtype: object

In [19]:
crime_df.head()

Primary Type,Location,Frequency
0,"(36.619446395, -91.686565684)",322.0
1,"(41.644585429, -87.616512829)",6.0
2,"(41.644589713, -87.61587983)",9.0
3,"(41.644604096, -87.610728247)",8.0
4,"(41.644606368, -87.606301217)",3.0


In [20]:
print(type(crime_df['Location'][0]))

<class 'tuple'>


In [21]:
crime_df.head()

Primary Type,Location,Frequency
0,"(36.619446395, -91.686565684)",322.0
1,"(41.644585429, -87.616512829)",6.0
2,"(41.644589713, -87.61587983)",9.0
3,"(41.644604096, -87.610728247)",8.0
4,"(41.644606368, -87.606301217)",3.0


In [22]:
crime_df = output_df.copy()

In [23]:
crime_df.head()

Primary Type,Location,Frequency
0,"(36.619446395, -91.686565684)",322.0
1,"(41.644585429, -87.616512829)",6.0
2,"(41.644589713, -87.61587983)",9.0
3,"(41.644604096, -87.610728247)",8.0
4,"(41.644606368, -87.606301217)",3.0


In [24]:
crime_df['Location'] = crime_df['Location'].apply(lambda x: tuple(map(float, x.strip('()').split(', '))))

In [25]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

# Split the coordinates column into two separate columns for latitude and longitude
crime_df[['latitude', 'longitude']] = pd.DataFrame(crime_df['Location'].tolist(), index=crime_df.index)
crime_df.drop('Location', axis=1, inplace=True)



In [26]:
# Split the dataset into training and testing sets
train_size = int(len(crime_df) * 0.8)
train_data, test_data = crime_df[:train_size], crime_df[train_size:]

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(train_data[['latitude', 'longitude']])
y_train = train_data['Frequency'].values
X_test = scaler.transform(test_data[['latitude', 'longitude']])
y_test = test_data['Frequency'].values

# Train a KNN model on the training data
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

# Predict the crime frequency score for new coordinates
new_coords = [(36.619446395, -91.686565684), (41.644585429, -87.616512829), (41.644589713, -87.61587983), (41.644604096, -87.610728247)]
X_new = scaler.transform(pd.DataFrame(new_coords, columns=['latitude', 'longitude']))
y_pred = knn.predict(X_new)

# Print the predicted scores for the new coordinates
print(y_pred)


[67.   5.6  6.4 13.6]


In [27]:
print(y_pred)

[67.   5.6  6.4 13.6]


In [28]:
crime_df.to_csv("crime_df.csv")

In [29]:
from sklearn.preprocessing import MinMaxScaler

# Assuming y_pred is your predicted values
scaler = MinMaxScaler(feature_range=(0, 100))
y_pred_scaled = scaler.fit_transform(y_pred.reshape(-1, 1))

In [30]:
print(y_pred_scaled)

[[100.        ]
 [  0.        ]
 [  1.3029316 ]
 [ 13.02931596]]


In [31]:
def predict(coord):
    X_new = scaler.transform(pd.DataFrame(new_coords, columns=['latitude', 'longitude']))
    y_pred = knn.predict(X_new)
    scaler1 = MinMaxScaler(feature_range=(0, 100))
    y_pred_scaled = scaler1.fit_transform(y_pred)
    print('predicted: ',y_pred)
    print('scaled: ',y_pred_scaled)
    return y_pred[0] # Return the first element of the array

In [33]:
import matplotlib.pyplot as plt

In [ ]:
print(y_pred)

In [ ]:
from sklearn.metrics import mean_absolute_error

y_pred = knn.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print('MAE:', mae)